<a href="https://colab.research.google.com/github/opsabarsec/NLP--film-genres-from-synopsis/blob/main/radix_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic assignment of genres from movie synopsis using supervised machine learning

## 1. Import libraries and load data

In [1]:
#packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# NLP libraries

from textblob import TextBlob, Word
import nltk
import re
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Deep learning libraries

from keras.models import Model
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Input, LSTM, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

In [2]:
# load data
train= pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')

## 2. DATA PREPARATION 

In [3]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# function for text cleaning 
def preprocess_text(text): #funsi yang berguna untuk  melakukan cleaning terhadap dataset
    text = text.lower() # lowercase
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text) #line breaks
    #text = re.sub(r"\'\xa0", " ", text) # xa0 Unicode representing spaces
    #text = re.sub('\s+', ' ', text) # one or more whitespace characters
    text = text.strip(' ') # spaces
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    #lemmatize and remove stopwords
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text) 
        
    return text

train['clean_plot'] = train['synopsis'].apply(lambda x: preprocess_text(x))
test['clean_plot'] = test['synopsis'].apply(lambda x: preprocess_text(x))

In [5]:
def lemma(text): # Lemmatization of cleaned body (avoid redundancy and improve the accuracy of analysis.)
        sent = TextBlob(text)
        tag_dict = {"J": 'a', 
                    "N": 'n', 
                    "V": 'v', 
                    "R": 'r'}
        words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        seperator=' '
        lemma = seperator.join(lemmatized_list) 
        return lemma

In [6]:
train['lemma'] = train['clean_plot'].apply(lambda x: lemma(x))
test['lemma'] = test['clean_plot'].apply(lambda x: lemma(x))

## 3. Variables preparation 

In [7]:

X_train = train['lemma']
X_test = test['lemma']    

### 3.1 Target variable one hot encoding

In [8]:
#apply the onehot transformation for the genres vector
y_train = train['genres']
one_hot = MultiLabelBinarizer() # encoder for the  tags 
y_onehot = one_hot.fit_transform(y_train.str.split(' ')) 
y_bin = pd.DataFrame(y_onehot, columns=one_hot.classes_ ) # transform it to Pandas object

In [9]:
# tokenize
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1 # Tambahan vocab_size

In [10]:
#fix max comment lenght 
maxlen = 200 # Diubah dari 100 jadi 500
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen, padding='post') # Ditambah padding post
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen, padding='post') # Ditambah padding post

## 4.The Model

In [11]:
#initialize parameters
inp = Input(shape=(maxlen, )) #maxlen defined earlier
embed_size = 128

In [12]:
# Neural network backbone
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=maxlen)) 
model.add(LSTM(64, return_sequences=True,name='lstm_layer'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(50, activation="relu"))
model.add(Dense(len(y_bin.columns), activation="softmax"))

In [13]:
# build the model

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [14]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
tensorboard = TensorBoard(log_dir="logs")
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

# train the model
batch_size = 64 # Diubah dari 16 ke 32
epochs = 10 # Diubah dari 3 ke 100
model.fit(X_t,y_onehot, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[tensorboard, early_stopping])


Epoch 1/10
514/514 [==============================] - 119s 223ms/step - loss: 0.2904 - accuracy: 0.2564 - val_loss: 0.2541 - val_accuracy: 0.3201
Epoch 2/10
514/514 [==============================] - 104s 203ms/step - loss: 0.2400 - accuracy: 0.3560 - val_loss: 0.2357 - val_accuracy: 0.3535
Epoch 3/10
514/514 [==============================] - 108s 211ms/step - loss: 0.2172 - accuracy: 0.4144 - val_loss: 0.2177 - val_accuracy: 0.4209
Epoch 4/10
514/514 [==============================] - 147s 285ms/step - loss: 0.1987 - accuracy: 0.4798 - val_loss: 0.2096 - val_accuracy: 0.4737
Epoch 5/10
514/514 [==============================] - 108s 210ms/step - loss: 0.1861 - accuracy: 0.5236 - val_loss: 0.2080 - val_accuracy: 0.4759
Epoch 6/10
514/514 [==============================] - 108s 211ms/step - loss: 0.1779 - accuracy: 0.5395 - val_loss: 0.2090 - val_accuracy: 0.4740
Epoch 7/10
514/514 [==============================] - 102s 199ms/step - loss: 0.1731 - accuracy: 0.5444 - val_loss: 0.2123 -

## 5.The prediction

In [15]:
filename = f'model-{epochs}-{batch_size}.h5'
model.save(filename)

In [16]:
from keras.models import load_model
model = load_model(filename)
y_pred = model.predict(X_te, batch_size=batch_size, verbose=1)
print(y_pred.shape)

83/83 [==============================] - 4s 45ms/step
(5250, 19)


In [17]:
df_probs_all = pd.DataFrame(y_pred,columns=y_bin.columns)
df_probs_all.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.211077,0.133561,0.047795,0.019449,0.041602,0.002951,0.002324,0.021603,0.049286,1.432115e-04,0.072223,0.005128,0.001460,0.007576,0.004811,0.339617,0.033342,0.001598,0.004452
1,0.262626,0.083559,0.021862,0.006284,0.020881,0.006270,0.017098,0.074003,0.025360,1.495468e-04,0.051692,0.002842,0.001333,0.010032,0.003333,0.317029,0.085774,0.006564,0.003309
2,0.000200,0.000073,0.000258,0.000053,0.001117,0.000058,0.987758,0.007289,0.000051,8.898140e-08,0.000091,0.000043,0.001225,0.000034,0.000125,0.000134,0.000103,0.001379,0.000008
3,0.024230,0.015417,0.015358,0.006250,0.030112,0.001915,0.000272,0.023624,0.033597,9.234443e-05,0.702886,0.000595,0.000709,0.016366,0.003642,0.076614,0.047482,0.000059,0.000780
4,0.035631,0.002863,0.000097,0.000188,0.029882,0.271074,0.000209,0.219279,0.001514,9.810552e-03,0.024140,0.000100,0.000141,0.073860,0.012481,0.003131,0.309481,0.000497,0.005619


In [18]:
def top_5_predictions(df):
    N = 5
    cols = df.columns[:-1].tolist()
    a = df[cols].to_numpy().argsort()[:, :-N-1:-1]
    c = np.array(cols)[a]
    d = df[cols].to_numpy()[np.arange(a.shape[0])[:, None], a]
    df1 = pd.DataFrame(c).rename(columns=lambda x : f'max_{x+1}_col')

    predicted_genres = df1["max_1_col"] + ', ' + df1["max_2_col"]+ ', ' +df1["max_3_col"]+ ', ' + df1["max_4_col"]+ ', '+df1["max_5_col"]
    return predicted_genres

In [19]:
pred_gen = top_5_predictions(df_probs_all)

In [20]:
submission = pd.DataFrame(data= {'movie_id':test.movie_id,'predicted_genres':pred_gen})

In [21]:
submission.head()

,movie_id,predicted_genres
0,10827,"Sci-Fi, Action, Adventure, Horror, Fantasy"
1,51768,"Sci-Fi, Action, Thriller, Adventure, Drama"
2,36064,"Documentary, Drama, War, Musical, Comedy"
3,33763,"Horror, Sci-Fi, Thriller, Fantasy, Comedy"
4,1146,"Thriller, Crime, Drama, Mystery, Action"


In [22]:
submission.to_csv('submission.csv',index=False)

In [23]:
new_synopsis_input = [input("Input the synopsis")]
synopsis = new_synopsis_input
new_synopsis_input = tokenizer.texts_to_sequences(new_synopsis_input)
new_synopsis_input = pad_sequences(new_synopsis_input, padding='post', maxlen=maxlen)
predict = model.predict(new_synopsis_input, batch_size=batch_size, verbose=1)
print(predict.shape)

result = pd.DataFrame(predict,columns=y_bin.columns)
print("The synopsis you entered is :\n", synopsis)
print("Genres : ", top_5_predictions(result))

1/1 [==============================] - 0s 29ms/step
(1, 19)
The synopsis you entered is :
 ['']
Genres :  0    Drama, Comedy, Adventure, Action, Romance
dtype: object
